In [ ]:
! python main.py --task celltype_GRN --data_file data/500_STRING_hESC/data.csv --net_file data/500_STRING_hESC/label.csv --setting new --alpha 100 --beta 1 --n_epoch 90 --save_name out
!python main.py --task non_celltype_GRN --data_file data/500_ChIP-seq_hESC/data.csv --net_file data/500_ChIP-seq_hESC/label.csv --setting new --alpha 0.1 --beta 0.01 --n_epochs 150  --save_name out

In [ ]:
# Calculate EPR values
import pandas as pd

output = pd.read_csv('../demo_data/GRN_inference/output/500_STRING_hESC_demo_output.tsv', sep='\t')
output['EdgeWeight'] = abs(output['EdgeWeight'])
output = output.sort_values('EdgeWeight', ascending=False)
label = pd.read_csv('../demo_data/GRN_inference/input//500_STRING_hESC/label.csv')
TFs = set(label['Gene1'])
Genes = set(label['Gene1']) | set(label['Gene2'])
output = output[output['Gene1'].apply(lambda x: x in TFs)]
output = output[output['Gene2'].apply(lambda x: x in Genes)]
label_set = set(label['Gene1'] + '|' + label['Gene2'])
output = output.iloc[:len(label_set)]
len(set(output['Gene1'] + '|' + output['Gene2']) & label_set) / (
            len(label_set) ** 2 / (len(TFs) * len(Genes) - len(TFs)))

In [ ]:
# Calculate AUPR ratio values
from sklearn.metrics import average_precision_score
import numpy as np
import pandas as pd

output = pd.read_csv('../demo_data/GRN_inference/output/500_STRING_hESC_demo_output.tsv', sep='\t')
output['EdgeWeight'] = abs(output['EdgeWeight'])
output = output.sort_values('EdgeWeight', ascending=False)
label = pd.read_csv('../demo_data/GRN_inference/input/500_STRING_hESC/label.csv')
TFs = set(label['Gene1'])
Genes = set(label['Gene1']) | set(label['Gene2'])
output = output[output['Gene1'].apply(lambda x: x in TFs)]
output = output[output['Gene2'].apply(lambda x: x in Genes)]
label_set = set(label['Gene1'] + label['Gene2'])
preds, labels, randoms = [], [], []
res_d = {}
l = []
p = []
for item in (output.to_dict('records')):
    res_d[item['Gene1'] + item['Gene2']] = item['EdgeWeight']
for item in (set(label['Gene1'])):
    for item2 in set(label['Gene1']) | set(label['Gene2']):
        if item + item2 in label_set:
            l.append(1)
        else:
            l.append(0)
        if item + item2 in res_d:
            p.append(res_d[item + item2])
        else:
            p.append(-1)
average_precision_score(l, p) / np.mean(l)